In [1]:
# Create Variables
$Password = "Passw0rd!"
$UserName = "testuser" # Can be also domain\testuser
$BasePath = "C:\GIT\Presentations\PowerShellUserGroup-2020-11-05"
$KeyPath = Join-Path -Path $BasePath -ChildPath "AES.key"
$PasswordPath = Join-Path -Path $BasePath -ChildPath "Password.txt"
$CLIPathIntegrated = Join-Path -Path $BasePath -ChildPath "CLIIntegrated.xml"
$CLIPathPSFramework = Join-Path -Path $BasePath -ChildPath "CLIPSFramework.xml"

$Password
$UserName
$BasePath
$KeyPath
$PasswordPath
$CLIPathIntegrated 
$CLIPathPSFramework

# PowerShell User Group innsalzach 05.11.2020

## Handling and managing Secrets using PowerShell

## Basics of PowerShell Credentials

If you want to create a secret you need to things:  

- a UserName which is plain text (you can replace UserName with anything else)
- the password (needs to be a secure string)(the password could also be an API KEY or something else you want to store as secure string)

## Passwords in an interactive Shell

You can also enter the Password like this:

Problem: What do you do if this is used in a pipeline or scheduled task for example?

Only applicable for interactive scripts.

In [ ]:
# Interactive Password
$InteractivePassword = Read-Host -AsSecureString -Prompt "Please Enter Password"
$InteractivePassword

## If you want to use the PSCredential Object only on this computer use the following example:

### Create a secure string

In [ ]:
# Create a secure string
$SecPassword = ConvertTo-SecureString -String $Password -AsPlainText -Force

Let's look at the return value

In [ ]:
# Returnvalue
$SecPassword

Get The password again in clear Text

In [ ]:

ConvertFrom-SecureString -SecureString $SecPassword -AsPlainText

### Create a PSCredential Object using a securestring obejct

**Note**: The SecPassword Variable has to be always of type System.Security.SecureString. Please do not put a string an Argument to the PSCredential Object.

**Reference in the Microsoft Docs**: [PSCredential Object](https://docs.microsoft.com/en-us/dotnet/api/system.management.automation.pscredential?view=powershellsdk-7.0.0)

In [ ]:

$splat = @{
    TypeName = "System.Management.Automation.PSCredential"
    ArgumentList = $UserName, $SecPassword
}

$PSCred = New-Object @splat 
$PSCred

Create a PSCredential Object in an interactive way

In [ ]:

$Cred = Get-Credential -Message "Please enter your password"

If you want to reuse that credential you can export it to a file in an encrypted way

In [ ]:

# PowerShell integrated way
$Cred | Export-CliXML -Path $CLIPathIntegrated

# PSFramewor way
$Cred | Export-PSFClixml -Path $CLIPathPSFramework

Import the XML Files whenever neeeded

**Note:** This can only be done on the same machine

In [ ]:

# Import Credential using the integrated way
Import-Clixml -Path $CLIPathIntegrated

# Import Credential using PSFramework
Import-PSFClixml -Path $CLIPathPSFramework

## If you want to share a PSCredential with remote Computers also please use a key.  

**Note**: You can use that also if you want to use It only on one computer.

## Create a Key File

**Note**: If you want to make that key accessible to more computers store It somewhere safe on a fileshare. (Take care of the right ACLs)

In [ ]:

$KeyPath = Join-Path -Path $BasePath -ChildPath "AES.key"
$Key = New-Object -TypeName byte[] 16
[Security.Cryptography.RNGCryptoServiceProvider]::Create().GetBytes($Key)
$key | Out-File -FilePath $KeyPath

###  Create a SecureString and encrypt It with the key

In [ ]:

$key = Get-Content -Path $KeyPath
$EncPassword = ConvertTo-SecureString -String $Password -AsPlainText -Force
$EncPassword | ConvertFrom-SecureString -Key $key | Out-File -FilePath $PasswordPath

Reminder: Open Password File

### Create the PSCredential Object

In [ ]:

$key = Get-Content -Path $KeyPath
$pwd = Get-Content -Path $PasswordPath | ConvertTo-SecureString -Key $key
$PSCred = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $UserName, $pwd

TODO: KeePass, Azure KeyVault, LastPass, Secret Management

## Housekeeping

In [2]:

Remove-Item -Path $KeyPath -Force
Remove-Item -Path $PasswordPath -Force
Remove-Item -Path $CLIPathIntegrated -Force
Remove-Item -Path $CLIPathPSFramework -Force